In [ ]:
%pip install aiohttp

# Submission Monitoring

This notebook monitors the document submission process and sends alerts for failures or delays.


In [ ]:
# Import required libraries
import sys
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
import json

print("✓ Basic imports completed")

sys.path.append("../../../src")

# Import ConfigManager from framework
from framework.utils.config import ConfigManager

# Get parameters from unified workflow setup task values
environment = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="environment")
globals_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="globals_config"))
catalog = globals_config["catalog"]
bronze_schema = globals_config["bronze_schema_name"]
output_table = dbutils.jobs.taskValues.get(taskKey="streaming_document_submission", key="output_table")
notification_email = dbutils.widgets.get("notification_email")

print(f"✓ Configuration loaded for monitoring")


In [ ]:
# Monitor submission failures and delays
submission_table = f"{catalog}.{bronze_schema}.{output_table}"

# Check for failed submissions
failed_docs = spark.sql(f"""
    SELECT COUNT(*) as failed_count
    FROM {submission_table}
    WHERE submission_status = 'failed'
    AND DATE(submission_timestamp) = CURRENT_DATE()
""").collect()[0].failed_count

# Check for stuck submissions (submitted > 1 hour ago, still in submitted status)
stuck_docs = spark.sql(f"""
    SELECT COUNT(*) as stuck_count
    FROM {submission_table}
    WHERE submission_status = 'submitted'
    AND submission_timestamp < current_timestamp() - INTERVAL 1 HOUR
""").collect()[0].stuck_count

print(f"📊 Monitoring Summary:")
print(f"  Failed today: {failed_docs}")
print(f"  Stuck submissions: {stuck_docs}")

if failed_docs > 0 or stuck_docs > 0:
    print("⚠️ Issues detected - would send notification in production")
    
print("✅ Monitoring completed")
